Here are all the codes that have been used to access the API for English, Scottish, and Welsh rainfall, in order to retrieve station ID, coordinates, and rainfall amount. This data is then recorded in a CSV file. Additionally, the county name is extracted based on the coordinates using the OpenCage package. The code also averages out the rainfall amount for each county in a separate column. This page also contains a sort of evolutionary history of the code, detailing how it developed over time into the final version that performs all these tasks.

### 1. England- Rainfall 

In [38]:
import requests

def get_rainfall_measurements():
    url = "http://environment.data.gov.uk/flood-monitoring/id/measures?parameter=rainfall"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        measurements = data['items']
        # Extract only measurements with data
        rainfall_info = []
        total_rainfall = 0  # Initialize total rainfall variable
        for measurement in measurements:
            if 'latestReading' in measurement:
                station_code = measurement.get('stationReference')
                rainfall_value = measurement['latestReading'].get('value')
                if rainfall_value is not None:
                    total_rainfall += rainfall_value  # Summing up the rainfall
                rainfall_info.append((station_code, rainfall_value))
        return rainfall_info, total_rainfall
    else:
        return "Error: Unable to fetch data", 0

rainfall_measurements, total_rainfall = get_rainfall_measurements()

# Print the total rainfall
print(f"Total Rainfall: {total_rainfall}")

# Now print each station's rainfall data
for station_code, rainfall in rainfall_measurements:
    print(f"Station Code: {station_code}, Rainfall: {rainfall}")


Total Rainfall: 55.99700000000013
Station Code: E7050, Rainfall: 0.0
Station Code: 0890TH, Rainfall: 0.0
Station Code: 3680, Rainfall: 0.0
Station Code: 3275, Rainfall: 0.2
Station Code: 3167, Rainfall: 0.6
Station Code: 3307, Rainfall: 0.2
Station Code: 3404, Rainfall: 0.2
Station Code: 3014, Rainfall: 0.2
Station Code: 3901, Rainfall: 0.2
Station Code: 3999, Rainfall: 0.0
Station Code: 1412, Rainfall: 0.4
Station Code: Not_Specified, Rainfall: 0.0
Station Code: Not_Specified, Rainfall: 0.04
Station Code: E13600, Rainfall: 0.0
Station Code: 242534TP, Rainfall: 0.0
Station Code: E8520, Rainfall: 0.0
Station Code: 276316TP, Rainfall: 0.0
Station Code: E10130, Rainfall: 0.0
Station Code: E24238, Rainfall: 0.0
Station Code: 000181TP, Rainfall: 0.0
Station Code: 238605TP, Rainfall: 0.0
Station Code: 52210, Rainfall: 0.0
Station Code: 47161, Rainfall: 0.1
Station Code: E1039, Rainfall: 0.0
Station Code: E8770, Rainfall: 0.0
Station Code: 244569TP, Rainfall: 0.0
Station Code: 595273, Rainfal

<br>

### 2. England- Coordinates

In [39]:
import requests

def fetch_station_data():
    url = 'https://environment.data.gov.uk/flood-monitoring/id/stations?parameter=rainfall'
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if 'items' in data:
            for station in data['items']:
                latitude = station.get('lat')
                longitude = station.get('long')
                print(f"Station: {station.get('notation', 'Unknown')}, Latitude: {latitude}, Longitude: {longitude}")
        else:
            print('No data available')
    else:
        print(f"Error fetching data: {response.status_code}")

fetch_station_data()


Station: E7050, Latitude: 52.186277, Longitude: -1.171327
Station: 4163, Latitude: 52.986245, Longitude: -1.163701
Station: 0890TH, Latitude: 51.692891, Longitude: -1.672984
Station: E1310, Latitude: 52.493886, Longitude: -0.649292
Station: 3680, Latitude: 52.73152, Longitude: -0.995167
Station: 3275, Latitude: 52.635078, Longitude: -1.944539
Station: 3167, Latitude: 52.419334, Longitude: -1.990391
Station: 3307, Latitude: 53.196879, Longitude: -1.901908
Station: 3404, Latitude: 53.287863, Longitude: -1.579192
Station: 3014, Latitude: 52.886657, Longitude: -2.182004
Station: 3901, Latitude: 53.305797, Longitude: -1.088289
Station: 3999, Latitude: 52.564065, Longitude: -1.199634
Station: 1412, Latitude: 52.47914, Longitude: -2.975413
Station: Not_Specified, Latitude: 50.778033, Longitude: -2.848812
Station: E13600, Latitude: 50.880106, Longitude: -1.558591
Station: 242534TP, Latitude: 51.807828, Longitude: -0.03963
Station: E8520, Latitude: 51.039914, Longitude: -0.097897
Station: 27631

<br>

### 3. England- Both Coordinates and Rainfall (takes long)

In [ ]:
import requests

def fetch_station_data_with_measurements():
    url = 'https://environment.data.gov.uk/flood-monitoring/id/stations?parameter=rainfall'
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if 'items' in data:
            for station in data['items']:
                # Fetch station details
                latitude = station.get('lat')
                longitude = station.get('long')
                station_id = station.get('notation', 'Unknown')

                # Fetch latest measurement for each station
                if 'measures' in station and station['measures']:
                    measure_url = station['measures'][0]['@id']
                    measure_response = requests.get(measure_url + '/readings?latest')
                    if measure_response.status_code == 200:
                        measure_data = measure_response.json()
                        if 'items' in measure_data and measure_data['items']:
                            latest_reading = measure_data['items'][0]
                            rainfall_value = latest_reading.get('value')
                            print(f"Station ID: {station_id}, Latitude: {latitude}, Longitude: {longitude}, Rainfall Measurement (mm): {rainfall_value}")
                        else:
                            print(f"Station ID: {station_id}, Latitude: {latitude}, Longitude: {longitude}, No recent measurement data available")
                    else:
                        print(f"Error fetching measurement data for station {station_id}")
                else:
                    print(f"Station ID: {station_id}, Latitude: {latitude}, Longitude: {longitude}, No measurement data available")
        else:
            print('No station data available')
    else:
        print(f"Error fetching station data: {response.status_code}")

fetch_station_data_with_measurements()


<br>

### 4. Scotland- Coordinates

In [55]:
import requests

def get_rain_stations_coordinates():
    base_url = "https://www2.sepa.org.uk/rainfall"
    endpoint = f"{base_url}/api/Stations"
    
    response = requests.get(endpoint)
    if response.status_code != 200:
        return "Failed to retrieve data from the API"

    stations_data = response.json()
    coordinates = []

    for station in stations_data:
        latitude = station.get('station_latitude')
        longitude = station.get('station_longitude')
        coordinates.append((latitude, longitude))

    return coordinates

coordinates = get_rain_stations_coordinates()
print(coordinates)


[('55.85329633', '-2.387448356'), ('57.48021805', '-3.205975095'), ('57.26272622', '-5.005979754'), ('57.24230294', '-2.717123004'), ('58.34717676', '-4.644621014'), ('57.28490502', '-6.187008123'), ('55.66964373', '-4.413260863'), ('55.35009103', '-5.717857841'), ('56.06640057', '-3.371633207'), ('55.66308049', '-4.757486092'), ('57.22063972', '-3.651932222'), ('56.11221933', '-4.342412217'), ('56.31678388', '-4.112254665'), ('56.47302976', '-3.904678161'), ('57.50564634', '-2.782377895'), ('55.66347112', '-4.051617294'), ('55.84727645', '-2.114611023'), ('57.86243941', '-5.318746664'), ('55.7838821', '-3.394584951'), ('56.22101614', '-3.042679056'), ('56.06523441', '-4.333343318'), ('55.73428872', '-4.5878755'), ('57.25007263', '-2.0473803'), ('56.48264025', '-2.905576117'), ('56.09528478', '-3.911816854'), ('55.98678855', '-4.056528766'), ('55.65035093', '-3.543765278'), ('58.16554218', '-6.740713858'), ('55.52935031', '-4.326283061'), ('57.61017023', '-3.222796593'), ('55.59164323'

### 5. Scotland rainfall measurement with time

In [1]:
import requests

# Function to fetch station data with coordinates for Scotland
def fetch_station_data_sco():
    sco_url = "https://www2.sepa.org.uk/rainfall/api/Stations"
    sco_response = requests.get(sco_url)
    if sco_response.status_code == 200:
        sco_stations = sco_response.json()
        return [station['station_no'] for station in sco_stations]
    return []

# Function to fetch latest hourly rainfall data for Scotland
def get_rainfall_data_sco(station_id):
    sco_url = f"https://www2.sepa.org.uk/rainfall/api/Hourly/{station_id}?all=true"
    sco_response = requests.get(sco_url)
    if sco_response.status_code == 200 and sco_response.json():
        return sco_response.json()[-1]
    return None

# Fetch station IDs
station_ids = fetch_station_data_sco()

# Iterate through stations and fetch latest rainfall data
for station_id in station_ids:
    latest_rainfall = get_rainfall_data_sco(station_id)
    if latest_rainfall:
        print(f"Station ID: {station_id}, Latest Rainfall: {latest_rainfall['Value']} at {latest_rainfall['Timestamp']}")


Station ID: 15018, Latest Rainfall: 0 at 19/03/2024 13:00:00
Station ID: 234150, Latest Rainfall: 0 at 19/03/2024 14:00:00
Station ID: 115301, Latest Rainfall: 0 at 19/03/2024 14:00:00
Station ID: 234170, Latest Rainfall: 0.2 at 19/03/2024 13:00:00
Station ID: 234151, Latest Rainfall: 0 at 19/03/2024 14:00:00
Station ID: 115304, Latest Rainfall: 0 at 19/03/2024 13:00:00
Station ID: 115500, Latest Rainfall: 0 at 19/03/2024 14:00:00
Station ID: 115501, Latest Rainfall: 0 at 19/03/2024 14:00:00
Station ID: 15198, Latest Rainfall: 0 at 19/03/2024 13:00:00
Station ID: 115506, Latest Rainfall: 0 at 19/03/2024 14:00:00
Station ID: 491811, Latest Rainfall: 0 at 19/03/2024 13:00:00
Station ID: 15163, Latest Rainfall: 0 at 19/03/2024 13:00:00
Station ID: 14964, Latest Rainfall: 0 at 19/03/2024 14:00:00
Station ID: 14952, Latest Rainfall: 0 at 19/03/2024 14:00:00
Station ID: 234161, Latest Rainfall: 0 at 19/03/2024 10:00:00
Station ID: 322556, Latest Rainfall: 0 at 19/03/2024 13:00:00
Station ID:

### 6. Wales- Rainfall and Coordinates (from Lauren)
#### But it collects river data as well, so we can't use it

In [ ]:
import pandas as pd
import csv
import requests

# Function to fetch rainfall measurements
def get_wales_rainfall_measurements():
    walesurl = 'https://api.naturalresources.wales/rivers-and-seas/v1/api/StationData'
    headers = {'Ocp-Apim-Subscription-Key': '413a14f470f64b70a010cfa3b4ed6a79'}
    walesresponse = requests.get(walesurl, headers=headers)
    if walesresponse.status_code == 200:
        walesdata = walesresponse.json()
        return walesdata
    else:
        return []

# Call the function and store the returned data
wales_rainfall_measurements = get_wales_rainfall_measurements()

# If there are no measurements, exit the script
if not wales_rainfall_measurements:
    print("No measurements found.")
else:
    # Specify the name of your CSV file
    filename = "wales_rainfall_measurements.csv"

    # Open a file to write, 'w' mode is for writing, newline and encoding specified for file format
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        # Create a CSV writer object
        writer = csv.writer(file)

        # Writing the header row of the CSV file
        writer.writerow(["Station Name", "Latitude", "Longitude", "Measurement Value (mm)"])

        # Iterating through each measurement in the data
        for measurement in wales_rainfall_measurements:
            # Extracting station name, latitude, longitude, and rainfall value
            wales_station_name = measurement.get('nameEN')
            wales_coordinates = measurement.get('coordinates')
            wales_latitude = wales_coordinates.get('latitude') if wales_coordinates else None
            wales_longitude = wales_coordinates.get('longitude') if wales_coordinates else None
            wales_parameters = measurement.get('parameters')
            wales_rainfall_value = wales_parameters[0].get('latestValue') if wales_parameters else None

            # Writing the data row to the CSV file
            writer.writerow([wales_station_name, wales_latitude, wales_longitude, wales_rainfall_value])

    print(f"Data has been written to {filename}")

<br>

### 7. Wales- Rainfall and Coordinates (from Imran)
#### This one only collects rainfall data

In [20]:
import requests
import pandas as pd
from IPython.display import display

# API key should be kept secure
api_key = '413a14f470f64b70a010cfa3b4ed6a79'

# Function to fetch data from API
def get_wales_api_data(api_key):
    url = 'https://api.naturalresources.wales/rivers-and-seas/v1/api/StationData'
    headers = {'Ocp-Apim-Subscription-Key': api_key}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        return []

# Call the function and store the returned data
api_data = get_wales_api_data(api_key)

# Prepare data for DataFrame
data = []
for station in api_data:  # Ensure 'api_data' is the variable containing your API data
    latitude = station['coordinates']['latitude']
    longitude = station['coordinates']['longitude']
    location = station['location']
    
    # Find the 'Rainfall' parameter
    latest_value = None
    for parameter in station['parameters']:
        if parameter['paramNameEN'] == 'Rainfall':
            latest_value = parameter['latestValue']
            break

    # Only add to data if latest_value is found
    if latest_value is not None:
        data.append([latitude, longitude, location, latest_value])

# Convert list to DataFrame
df = pd.DataFrame(data, columns=["Latitude", "Longitude", "Location", "Latest Value (mm)"])
# Save DataFrame to CSV
df.to_csv("Wales.csv", index=False)

# Display the DataFrame
display(df)

,Latitude,Longitude,Location,Latest Value (mm)
0,53.115798,-3.296682,1000,0.0
1,52.777033,-3.367889,1001,0.0
2,53.139140,-3.677650,1002,0.0
3,53.219740,-3.209041,1003,0.0
4,53.168868,-3.570317,1004,0.0
...,...,...,...,...
145,52.879647,-3.159123,1145,0.0
146,52.320487,-3.057786,1146,0.0
147,52.507280,-3.170619,1147,0.0
148,52.757895,-3.460426,1148,0.0


### 7.1 Display Wales API

In [ ]:
import requests
import pandas as pd
from IPython.display import JSON

# Function to fetch data from API
def get_wales_api_data():
    url = 'https://api.naturalresources.wales/rivers-and-seas/v1/api/StationData'
    headers = {'Ocp-Apim-Subscription-Key': '413a14f470f64b70a010cfa3b4ed6a79'}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Call the function and store the returned data
api_data = get_wales_api_data()

# Check if data was returned
if api_data is None:
    print("Failed to retrieve data or no data available.")
else:
    # Display the complete JSON response
    display(JSON(api_data))


<br>

### 8. Combines all the API calls and saves the data in a CSV

In [59]:
import requests
import csv

# Function to fetch station data with coordinates for England
def fetch_station_data_eng():
    url = 'https://environment.data.gov.uk/flood-monitoring/id/stations?parameter=rainfall'
    response = requests.get(url)
    eng_station_data = {}
    if response.status_code == 200:
        data = response.json()
        for station in data['items']:
            eng_station_data[station.get('notation')] = {
                'latitude': station.get('lat'),
                'longitude': station.get('long')
            }
    return eng_station_data

# Function to fetch rainfall measurements for England
def get_rainfall_data_eng():
    eng_url = "http://environment.data.gov.uk/flood-monitoring/id/measures?parameter=rainfall"
    eng_response = requests.get(eng_url)
    if eng_response.status_code == 200:
        eng_data = eng_response.json()
        return eng_data['items']
    return []

# Function to fetch station data with coordinates for Scotland
def fetch_station_data_sco():
    sco_url = "https://www2.sepa.org.uk/rainfall/api/Stations"
    sco_response = requests.get(sco_url)
    sco_station_data = {}
    if sco_response.status_code == 200:
        sco_stations = sco_response.json()
        for station in sco_stations:
            sco_station_data[station['station_no']] = {
                'latitude': station.get('station_latitude'),
                'longitude': station.get('station_longitude')
            }
    return sco_station_data

# Function to fetch latest hourly rainfall data for Scotland
def get_rainfall_data_sco(station_id):
    sco_url = f"https://www2.sepa.org.uk/rainfall/api/Hourly/{station_id}?all=true"
    sco_response = requests.get(sco_url)
    if sco_response.status_code == 200 and sco_response.json():
        return sco_response.json()[-1]
    return None

# Function to fetch station data with rainfall measurements for Wales
def get_wales_rainfall_data(api_key):
    url = 'https://api.naturalresources.wales/rivers-and-seas/v1/api/StationData'
    headers = {'Ocp-Apim-Subscription-Key': api_key}
    response = requests.get(url, headers=headers)
    wales_rainfall_data = []
    if response.status_code == 200:
        wales_data = response.json()
        for station in wales_data:
            station_id = station['location']
            latitude = station['coordinates']['latitude']
            longitude = station['coordinates']['longitude']
            
            # Find the 'Rainfall' parameter
            rainfall = None
            for parameter in station['parameters']:
                if parameter['paramNameEN'] == 'Rainfall':
                    rainfall = parameter['latestValue']
                    break

            # Add the station data to the list if rainfall data is found
            if rainfall is not None:
                wales_rainfall_data.append({
                    'station_id': station_id,
                    'rainfall': rainfall,
                    'latitude': latitude,
                    'longitude': longitude
                })
    return wales_rainfall_data

#  API key should be kept secure
api_key = '413a14f470f64b70a010cfa3b4ed6a79'

# Fetching and processing data for England and Scotland
eng_station_coordinates = fetch_station_data_eng()
eng_rainfall_data = get_rainfall_data_eng()
sco_station_coordinates = fetch_station_data_sco()
sco_rainfall_data = {station_id: get_rainfall_data_sco(station_id) for station_id in sco_station_coordinates}

# Fetching and processing data for Wales
wales_rainfall_data = get_wales_rainfall_data(api_key)

# Combine the data
combined_data = []

# Process and combine England data
for measurement in eng_rainfall_data:
    station_id = measurement.get('stationReference')
    rainfall = measurement.get('latestReading', {}).get('value')
    coordinates = eng_station_coordinates.get(station_id, {'latitude': None, 'longitude': None})
    combined_data.append([station_id, rainfall, coordinates['latitude'], coordinates['longitude'], 'England'])

# Process and combine Scotland data
for station_id, latest_hourly_data in sco_rainfall_data.items():
    if latest_hourly_data:
        rainfall = latest_hourly_data['Value']
        coordinates = sco_station_coordinates[station_id]
        combined_data.append([station_id, rainfall, coordinates['latitude'], coordinates['longitude'], 'Scotland'])

# Process and combine Wales data
for station_data in wales_rainfall_data:
    combined_data.append([
        station_data['station_id'],
        station_data['rainfall'],
        station_data['latitude'],
        station_data['longitude'],
        'Wales'
    ])

# Write to CSV
filename = "final_combined_rainfall_measurements.csv"
with open(filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Station ID", "Rainfall Measurement (mm)", "Latitude", "Longitude", "Country"])
    writer.writerows(combined_data)

print(f"Data has been written to {filename}")


Data has been written to final_combined_rainfall_measurements.csv


### 9. Exludes stations where there are no cordinates data. Otherwise, does similar job as the previous (7) one

In [27]:
import requests
import csv

# Function to fetch station data with coordinates for England
def fetch_station_data_eng():
    url = 'https://environment.data.gov.uk/flood-monitoring/id/stations?parameter=rainfall'
    response = requests.get(url)
    eng_station_data = {}
    if response.status_code == 200:
        data = response.json()
        for station in data['items']:
            if station.get('lat') is not None and station.get('long') is not None:
                eng_station_data[station.get('notation')] = {
                    'latitude': station.get('lat'),
                    'longitude': station.get('long')
                }
    return eng_station_data

# Function to fetch rainfall measurements for England
def get_rainfall_data_eng():
    eng_url = "http://environment.data.gov.uk/flood-monitoring/id/measures?parameter=rainfall"
    eng_response = requests.get(eng_url)
    if eng_response.status_code == 200:
        eng_data = eng_response.json()
        return eng_data['items']
    return []

# Function to fetch station data with coordinates for Scotland
def fetch_station_data_sco():
    sco_url = "https://www2.sepa.org.uk/rainfall/api/Stations"
    sco_response = requests.get(sco_url)
    sco_station_data = {}
    if sco_response.status_code == 200:
        sco_stations = sco_response.json()
        for station in sco_stations:
            if station.get('station_latitude') is not None and station.get('station_longitude') is not None:
                sco_station_data[station['station_no']] = {
                    'latitude': station.get('station_latitude'),
                    'longitude': station.get('station_longitude')
                }
    return sco_station_data

# Function to fetch latest hourly rainfall data for Scotland
def get_rainfall_data_sco(station_id):
    sco_url = f"https://www2.sepa.org.uk/rainfall/api/Hourly/{station_id}?all=true"
    sco_response = requests.get(sco_url)
    if sco_response.status_code == 200 and sco_response.json():
        return sco_response.json()[-1]
    return None

# Function to fetch station data with rainfall measurements for Wales
def get_wales_rainfall_data(api_key):
    url = 'https://api.naturalresources.wales/rivers-and-seas/v1/api/StationData'
    headers = {'Ocp-Apim-Subscription-Key': api_key}
    response = requests.get(url, headers=headers)
    wales_rainfall_data = []
    if response.status_code == 200:
        wales_data = response.json()
        for station in wales_data:
            if station['coordinates']['latitude'] is not None and station['coordinates']['longitude'] is not None:
                station_id = station['location']
                latitude = station['coordinates']['latitude']
                longitude = station['coordinates']['longitude']
                
                # Find the 'Rainfall' parameter
                rainfall = None
                for parameter in station['parameters']:
                    if parameter['paramNameEN'] == 'Rainfall':
                        rainfall = parameter['latestValue']
                        break

                # Add the station data to the list if rainfall data is found
                if rainfall is not None:
                    wales_rainfall_data.append({
                        'station_id': station_id,
                        'rainfall': rainfall,
                        'latitude': latitude,
                        'longitude': longitude
                    })
    return wales_rainfall_data

# API key should be kept secure
api_key = '413a14f470f64b70a010cfa3b4ed6a79'

# Fetching and processing data for England and Scotland
eng_station_coordinates = fetch_station_data_eng()
eng_rainfall_data = get_rainfall_data_eng()
sco_station_coordinates = fetch_station_data_sco()
sco_rainfall_data = {station_id: get_rainfall_data_sco(station_id) for station_id in sco_station_coordinates}

# Fetching and processing data for Wales
wales_rainfall_data = get_wales_rainfall_data(api_key)

# Combine the data
combined_data = []

# Process and combine England data
for measurement in eng_rainfall_data:
    station_id = measurement.get('stationReference')
    rainfall = measurement.get('latestReading', {}).get('value')
    if station_id in eng_station_coordinates:
        coordinates = eng_station_coordinates[station_id]
        combined_data.append([station_id, rainfall, coordinates['latitude'], coordinates['longitude'], 'England'])

# Process and combine Scotland data
for station_id, latest_hourly_data in sco_rainfall_data.items():
    if latest_hourly_data and station_id in sco_station_coordinates:
        rainfall = latest_hourly_data['Value']
        coordinates = sco_station_coordinates[station_id]
        combined_data.append([station_id, rainfall, coordinates['latitude'], coordinates['longitude'], 'Scotland'])

# Process and combine Wales data
for station_data in wales_rainfall_data:
    combined_data.append([
        station_data['station_id'],
        station_data['rainfall'],
        station_data['latitude'],
        station_data['longitude'],
        'Wales'
    ])

# Write to CSV
filename = "final_combined_rainfall_measurements.csv"
with open(filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Station ID", "Rainfall Measurement (mm)", "Latitude", "Longitude", "Country"])
    writer.writerows(combined_data)

print(f"Data has been written to {filename}")


Data has been written to final_combined_rainfall_measurements.csv


### 10. same as before but getting county names through geopy

In [1]:
import requests
import csv
from geopy.geocoders import Nominatim
import time
from datetime import datetime

# Function to fetch county name using Geopy
def get_county_name(latitude, longitude):
    try:
        geolocator = Nominatim(user_agent="geoapiExercises")
        location = geolocator.reverse((latitude, longitude), exactly_one=True)
        address = location.raw['address']
        return address.get('county', 'County not found')
    except:
        return 'County not found'

# Function to fetch station data with coordinates for England
def fetch_station_data_eng():
    url = 'https://environment.data.gov.uk/flood-monitoring/id/stations?parameter=rainfall'
    response = requests.get(url)
    eng_station_data = {}
    if response.status_code == 200:
        data = response.json()
        for station in data['items']:
            if station.get('lat') is not None and station.get('long') is not None:
                eng_station_data[station.get('notation')] = {
                    'latitude': station.get('lat'),
                    'longitude': station.get('long')
                }
    return eng_station_data

# Function to fetch rainfall measurements for England
def get_rainfall_data_eng():
    eng_url = "http://environment.data.gov.uk/flood-monitoring/id/measures?parameter=rainfall"
    eng_response = requests.get(eng_url)
    if eng_response.status_code == 200:
        eng_data = eng_response.json()
        return eng_data['items']
    return []

# Function to fetch station data with coordinates for Scotland
def fetch_station_data_sco():
    sco_url = "https://www2.sepa.org.uk/rainfall/api/Stations"
    sco_response = requests.get(sco_url)
    sco_station_data = {}
    if sco_response.status_code == 200:
        sco_stations = sco_response.json()
        for station in sco_stations:
            if station.get('station_latitude') is not None and station.get('station_longitude') is not None:
                sco_station_data[station['station_no']] = {
                    'latitude': station.get('station_latitude'),
                    'longitude': station.get('station_longitude')
                }
    return sco_station_data

# Function to fetch latest hourly rainfall data for Scotland
def get_rainfall_data_sco(station_id):
    sco_url = f"https://www2.sepa.org.uk/rainfall/api/Hourly/{station_id}?all=true"
    sco_response = requests.get(sco_url)
    if sco_response.status_code == 200 and sco_response.json():
        return sco_response.json()[-1]
    return None

# Function to fetch station data with rainfall measurements for Wales
def get_wales_rainfall_data(api_key):
    url = 'https://api.naturalresources.wales/rivers-and-seas/v1/api/StationData'
    headers = {'Ocp-Apim-Subscription-Key': api_key}
    response = requests.get(url, headers=headers)
    wales_rainfall_data = []
    if response.status_code == 200:
        wales_data = response.json()
        for station in wales_data:
            if station['coordinates']['latitude'] is not None and station['coordinates']['longitude'] is not None:
                station_id = station['location']
                latitude = station['coordinates']['latitude']
                longitude = station['coordinates']['longitude']
                
                # Find the 'Rainfall' parameter
                rainfall = None
                for parameter in station['parameters']:
                    if parameter['paramNameEN'] == 'Rainfall':
                        rainfall = parameter['latestValue']
                        break

                # Add the station data to the list if rainfall data is found
                if rainfall is not None:
                    wales_rainfall_data.append({
                        'station_id': station_id,
                        'rainfall': rainfall,
                        'latitude': latitude,
                        'longitude': longitude
                    })
    return wales_rainfall_data

# API key should be kept secure
api_key = '413a14f470f64b70a010cfa3b4ed6a79'

# Fetching and processing data for England, Scotland, and Wales
eng_station_coordinates = fetch_station_data_eng()
eng_rainfall_data = get_rainfall_data_eng()
sco_station_coordinates = fetch_station_data_sco()
sco_rainfall_data = {station_id: get_rainfall_data_sco(station_id) for station_id in sco_station_coordinates}
wales_rainfall_data = get_wales_rainfall_data(api_key)


# Combine the data
combined_data = []

# Process and combine England data
for measurement in eng_rainfall_data:
    station_id = measurement.get('stationReference')
    rainfall = measurement.get('latestReading', {}).get('value')
    coordinates = eng_station_coordinates.get(station_id, {'latitude': None, 'longitude': None})
    if coordinates['latitude'] is not None and coordinates['longitude'] is not None:
        combined_data.append([station_id, rainfall, coordinates['latitude'], coordinates['longitude'], 'England'])

# Process and combine Scotland data
for station_id, latest_hourly_data in sco_rainfall_data.items():
    if latest_hourly_data:
        rainfall = latest_hourly_data['Value']
        coordinates = sco_station_coordinates.get(station_id, {'latitude': None, 'longitude': None})
        if coordinates['latitude'] is not None and coordinates['longitude'] is not None:
            combined_data.append([station_id, rainfall, coordinates['latitude'], coordinates['longitude'], 'Scotland'])

# Process and combine Wales data
for station_data in wales_rainfall_data:
    station_id = station_data['station_id']
    rainfall = station_data['rainfall']
    latitude = station_data['latitude']
    longitude = station_data['longitude']
    if latitude is not None and longitude is not None:
        combined_data.append([station_id, rainfall, latitude, longitude, 'Wales'])


# Add geocoding for counties
for data_row in combined_data:
    latitude, longitude = data_row[2], data_row[3]
    if latitude and longitude:
        county = get_county_name(latitude, longitude)
        data_row.append(county)
    else:
        data_row.append('No coordinates')

    # To respect the usage limits of Nominatim, add a short sleep after each request
    time.sleep(.05)

# Write to CSV with the new 'County' column
filename = "final_combined_rainfall_measurements_with_county.csv"
with open(filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    header = ["Station ID", "Rainfall Measurement (mm)", "Latitude", "Longitude", "Country", "County"]
    writer.writerow(header)
    writer.writerows(combined_data)

current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Data has been written to {filename} and current time is {current_time}")


Data has been written to final_combined_rainfall_measurements_with_county.csv and current time is 2024-03-15 00:35:40


### 11. same as before but getting county names through opencage. Geopy gives many 'County Not FOund' message.

#### <span style="color:red"> Note: I am using free tier of opencage. so it can send only 2500 requests per day. currently we have about 1500 rows, which means we can only run this code once a day on free subscription. </span>

In [3]:
pip install opencage

  Obtaining dependency information for opencage from https://files.pythonhosted.org/packages/1e/c0/51441cbfe21146f5c153af3fa19f24f9621734af398f3da6f4e781adce68/opencage-2.4.0-py3-none-any.whl.metadata
  Obtaining dependency information for backoff>=2.2.1 from https://files.pythonhosted.org/packages/df/73/b6e24bd22e6720ca8ee9a85a0c4a2971af8497d8f3193fa05390cbd46e09/backoff-2.2.1-py3-none-any.whl.metadata

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import requests
import csv
from opencage.geocoder import OpenCageGeocode
import time
from datetime import datetime

# Function to fetch county name using OpenCage Geocoder
def get_county_name(latitude, longitude, opencage_api_key):
    try:
        geocoder = OpenCageGeocode(opencage_api_key)
        query = (latitude, longitude)
        results = geocoder.reverse_geocode(*query)

        if results and 'components' in results[0]:
            components = results[0]['components']
            if 'county' in components:
                return components['county']
            elif 'state_district' in components:
                return components['state_district']
            elif 'state' in components:
                return components['state']
            
        return 'County not found'
    except Exception as e:
        print(f"Error in geocoding: {e}")
        return 'County not found'


# Function to fetch station data with coordinates for England
def fetch_station_data_eng():
    url = 'https://environment.data.gov.uk/flood-monitoring/id/stations?parameter=rainfall'
    response = requests.get(url)
    eng_station_data = {}
    if response.status_code == 200:
        data = response.json()
        for station in data['items']:
            if station.get('lat') is not None and station.get('long') is not None:
                eng_station_data[station.get('notation')] = {
                    'latitude': station.get('lat'),
                    'longitude': station.get('long')
                }
    return eng_station_data

# Function to fetch rainfall measurements for England
def get_rainfall_data_eng():
    eng_url = "http://environment.data.gov.uk/flood-monitoring/id/measures?parameter=rainfall"
    eng_response = requests.get(eng_url)
    if eng_response.status_code == 200:
        eng_data = eng_response.json()
        return eng_data['items']
    return []

# Function to fetch station data with coordinates for Scotland
def fetch_station_data_sco():
    sco_url = "https://www2.sepa.org.uk/rainfall/api/Stations"
    sco_response = requests.get(sco_url)
    sco_station_data = {}
    if sco_response.status_code == 200:
        sco_stations = sco_response.json()
        for station in sco_stations:
            if station.get('station_latitude') is not None and station.get('station_longitude') is not None:
                sco_station_data[station['station_no']] = {
                    'latitude': station.get('station_latitude'),
                    'longitude': station.get('station_longitude')
                }
    return sco_station_data

# Function to fetch latest hourly rainfall data for Scotland
def get_rainfall_data_sco(station_id):
    sco_url = f"https://www2.sepa.org.uk/rainfall/api/Hourly/{station_id}?all=true"
    sco_response = requests.get(sco_url)
    if sco_response.status_code == 200 and sco_response.json():
        return sco_response.json()[-1]
    return None

# Function to fetch station data with rainfall measurements for Wales
def get_wales_rainfall_data(api_key):
    url = 'https://api.naturalresources.wales/rivers-and-seas/v1/api/StationData'
    headers = {'Ocp-Apim-Subscription-Key': api_key}
    response = requests.get(url, headers=headers)
    wales_rainfall_data = []
    if response.status_code == 200:
        wales_data = response.json()
        for station in wales_data:
            if station['coordinates']['latitude'] is not None and station['coordinates']['longitude'] is not None:
                station_id = station['location']
                latitude = station['coordinates']['latitude']
                longitude = station['coordinates']['longitude']
                
                # Find the 'Rainfall' parameter
                rainfall = None
                for parameter in station['parameters']:
                    if parameter['paramNameEN'] == 'Rainfall':
                        rainfall = parameter['latestValue']
                        break

                # Add the station data to the list if rainfall data is found
                if rainfall is not None:
                    wales_rainfall_data.append({
                        'station_id': station_id,
                        'rainfall': rainfall,
                        'latitude': latitude,
                        'longitude': longitude
                    })
    return wales_rainfall_data

# API key for OpenCage Geocoder
opencage_api_key = '26ea8244e3124756b142ca8a605395bd'

# Fetching and processing data for England, Scotland, and Wales
eng_station_coordinates = fetch_station_data_eng()
eng_rainfall_data = get_rainfall_data_eng()
sco_station_coordinates = fetch_station_data_sco()
sco_rainfall_data = {station_id: get_rainfall_data_sco(station_id) for station_id in sco_station_coordinates}
wales_rainfall_data = get_wales_rainfall_data('413a14f470f64b70a010cfa3b4ed6a79')  # Replace with the actual API key for Natural Resources Wales

# Combine the data
combined_data = []

# Process and combine England data
for measurement in eng_rainfall_data:
    station_id = measurement.get('stationReference')
    rainfall = measurement.get('latestReading', {}).get('value')
    coordinates = eng_station_coordinates.get(station_id, {'latitude': None, 'longitude': None})
    if coordinates['latitude'] is not None and coordinates['longitude'] is not None:
        combined_data.append([station_id, rainfall, coordinates['latitude'], coordinates['longitude'], 'England'])

# Process and combine Scotland data
for station_id, latest_hourly_data in sco_rainfall_data.items():
    if latest_hourly_data:
        rainfall = latest_hourly_data['Value']
        coordinates = sco_station_coordinates.get(station_id, {'latitude': None, 'longitude': None})
        if coordinates['latitude'] is not None and coordinates['longitude'] is not None:
            combined_data.append([station_id, rainfall, coordinates['latitude'], coordinates['longitude'], 'Scotland'])

# Process and combine Wales data
for station_data in wales_rainfall_data:
    station_id = station_data['station_id']
    rainfall = station_data['rainfall']
    latitude = station_data['latitude']
    longitude = station_data['longitude']
    if latitude is not None and longitude is not None:
        combined_data.append([station_id, rainfall, latitude, longitude, 'Wales'])

# Add geocoding for counties using OpenCage
for data_row in combined_data:
    latitude, longitude = data_row[2], data_row[3]
    if latitude and longitude:
        county = get_county_name(latitude, longitude, opencage_api_key)
        data_row.append(county)
    else:
        data_row.append('No coordinates')

    # To respect the usage limits of OpenCage, add a short sleep after each request
    time.sleep(1)

# Write to CSV with the new 'County' column
filename = "a_final_combined_rainfall_measurements_with_county.csv"
with open(filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    header = ["Station ID", "Rainfall Measurement (mm)", "Latitude", "Longitude", "Country", "County"]
    writer.writerow(header)
    writer.writerows(combined_data)

current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Data has been written to {filename} and current time is {current_time}")


Data has been written to a_final_combined_rainfall_measurements_with_county.csv and current time is 2024-03-15 01:46:34


## 12. Final version of the code

### same as previous but adds average rainfall for counties
#### <span style="color:red"> Note: It takes about 40 minutes for this code to execute</span>

In [24]:
import requests
import csv
from opencage.geocoder import OpenCageGeocode
import time
from datetime import datetime

# Function to fetch county name using OpenCage Geocoder
def get_county_name(latitude, longitude, opencage_api_key):
    try:
        geocoder = OpenCageGeocode(opencage_api_key)
        query = (latitude, longitude)
        results = geocoder.reverse_geocode(*query)

        if results and 'components' in results[0]:
            components = results[0]['components']
            if 'county' in components:
                return components['county']
            elif 'state_district' in components:
                return components['state_district']
            elif 'state' in components:
                return components['state']
            
        return 'County not found'
    except Exception as e:
        print(f"Error in geocoding: {e}")
        return 'County not found'

# Function to fetch station data with coordinates for England
def fetch_station_data_eng():
    url = 'https://environment.data.gov.uk/flood-monitoring/id/stations?parameter=rainfall'
    response = requests.get(url)
    eng_station_data = {}
    if response.status_code == 200:
        data = response.json()
        for station in data['items']:
            if station.get('lat') is not None and station.get('long') is not None:
                eng_station_data[station.get('notation')] = {
                    'latitude': station.get('lat'),
                    'longitude': station.get('long')
                }
    return eng_station_data

# Function to fetch rainfall measurements for England
def get_rainfall_data_eng():
    eng_url = "http://environment.data.gov.uk/flood-monitoring/id/measures?parameter=rainfall"
    eng_response = requests.get(eng_url)
    if eng_response.status_code == 200:
        eng_data = eng_response.json()
        return eng_data['items']
    return []




# Function to fetch station data with coordinates for Scotland
def fetch_station_data_sco():
    sco_url = "https://www2.sepa.org.uk/rainfall/api/Stations"
    sco_response = requests.get(sco_url)
    sco_station_data = {}
    if sco_response.status_code == 200:
        sco_stations = sco_response.json()
        for station in sco_stations:
            if station.get('station_latitude') is not None and station.get('station_longitude') is not None:
                sco_station_data[station['station_no']] = {
                    'latitude': station.get('station_latitude'),
                    'longitude': station.get('station_longitude')
                }
    return sco_station_data

# Function to fetch latest hourly rainfall data for Scotland
def get_rainfall_data_sco(station_id):
    sco_url = f"https://www2.sepa.org.uk/rainfall/api/Hourly/{station_id}?all=true"
    sco_response = requests.get(sco_url)
    if sco_response.status_code == 200 and sco_response.json():
        return sco_response.json()[-1]
    return None

# Function to fetch station data with rainfall measurements for Wales
def get_wales_rainfall_data(api_key):
    url = 'https://api.naturalresources.wales/rivers-and-seas/v1/api/StationData'
    headers = {'Ocp-Apim-Subscription-Key': api_key}
    response = requests.get(url, headers=headers)
    wales_rainfall_data = []
    if response.status_code == 200:
        wales_data = response.json()
        for station in wales_data:
            if station['coordinates']['latitude'] is not None and station['coordinates']['longitude'] is not None:
                station_id = station['location']
                latitude = station['coordinates']['latitude']
                longitude = station['coordinates']['longitude']
                
                # Find the 'Rainfall' parameter
                rainfall = None
                for parameter in station['parameters']:
                    if parameter['paramNameEN'] == 'Rainfall':
                        rainfall = parameter['latestValue']
                        break

                # Add the station data to the list if rainfall data is found
                if rainfall is not None:
                    wales_rainfall_data.append({
                        'station_id': station_id,
                        'rainfall': rainfall,
                        'latitude': latitude,
                        'longitude': longitude
                    })
    return wales_rainfall_data

# API key for OpenCage Geocoder
opencage_api_key = '26ea8244e3124756b142ca8a605395bd'

# Fetching and processing data for England, Scotland, and Wales
eng_station_coordinates = fetch_station_data_eng()
eng_rainfall_data = get_rainfall_data_eng()
sco_station_coordinates = fetch_station_data_sco()
sco_rainfall_data = {station_id: get_rainfall_data_sco(station_id) for station_id in sco_station_coordinates}
wales_rainfall_data = get_wales_rainfall_data('413a14f470f64b70a010cfa3b4ed6a79')  # Replace with the actual API key for Natural Resources Wales

# Combine the data
combined_data = []

# Process and combine England data
for measurement in eng_rainfall_data:
    station_id = measurement.get('stationReference')
    rainfall = measurement.get('latestReading', {}).get('value')
    coordinates = eng_station_coordinates.get(station_id, {'latitude': None, 'longitude': None})
    if coordinates['latitude'] is not None and coordinates['longitude'] is not None:
        combined_data.append([station_id, rainfall, coordinates['latitude'], coordinates['longitude'], 'England'])

# Process and combine Scotland data
for station_id, latest_hourly_data in sco_rainfall_data.items():
    if latest_hourly_data:
        rainfall = latest_hourly_data['Value']
        coordinates = sco_station_coordinates.get(station_id, {'latitude': None, 'longitude': None})
        if coordinates['latitude'] is not None and coordinates['longitude'] is not None:
            combined_data.append([station_id, rainfall, coordinates['latitude'], coordinates['longitude'], 'Scotland'])

# Process and combine Wales data
for station_data in wales_rainfall_data:
    station_id = station_data['station_id']
    rainfall = station_data['rainfall']
    latitude = station_data['latitude']
    longitude = station_data['longitude']
    if latitude is not None and longitude is not None:
        combined_data.append([station_id, rainfall, latitude, longitude, 'Wales'])

# Add geocoding for counties using OpenCage
for data_row in combined_data:
    latitude, longitude = data_row[2], data_row[3]
    if latitude and longitude:
        county = get_county_name(latitude, longitude, opencage_api_key)
        data_row.append(county)
    else:
        data_row.append('No coordinates')

    # To respect the usage limits of OpenCage, add a short sleep after each request
    time.sleep(1)

# Calculate average rainfall for each county
county_rainfall = {}
for data_row in combined_data:
    county = data_row[5]
    rainfall = float(data_row[1]) if data_row[1] else 0
    if county not in county_rainfall:
        county_rainfall[county] = {'total_rainfall': 0, 'count': 0}
    county_rainfall[county]['total_rainfall'] += rainfall
    county_rainfall[county]['count'] += 1

for county, data in county_rainfall.items():
    average_rainfall = round(data['total_rainfall'] / data['count'], 2) if data['count'] > 0 else 0
    county_rainfall[county]['average_rainfall'] = average_rainfall


# Append the average rainfall to the data rows
for data_row in combined_data:
    county = data_row[5]
    average_rainfall = county_rainfall[county].get('average_rainfall', 0)
    data_row.append(average_rainfall)

# Write to CSV with the new 'County Avg Rainfall' column
filename = "rainfall_measurements_with_county_and_avg.csv"
with open(filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    header = ["Station ID", "Rainfall Measurement (mm)", "Latitude", "Longitude", "Country", "County", "County Avg Rainfall (mm)"]
    writer.writerow(header)
    writer.writerows(combined_data)

current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Data has been written to {filename} and current time is {current_time}")


Data has been written to rainfall_measurements_with_county_and_avg.csv and current time is 2024-03-19 23:44:32


### This code take an existing CSV, which has station id and coordinates and county names and, and writes the rainfall amount from the API to the csv file

In [87]:
import requests
import csv
from datetime import datetime

# # Function to fetch station data with coordinates for England
# def fetch_station_data_eng():
#     url = 'https://environment.data.gov.uk/flood-monitoring/id/stations?parameter=rainfall'
#     response = requests.get(url)
#     eng_station_data = {}
#     if response.status_code == 200:
#         data = response.json()
#         for station in data['items']:
#             if station.get('latitude') is not None and station.get('longitude') is not None:
#                 eng_station_data[station.get('notation')] = {
#                     'latitude': station.get('latitude'),
#                     'longitude': station.get('longitude')
#                 }
#     return eng_station_data


# Function to fetch station data with coordinates for England
def fetch_station_data_eng():
    url = 'https://environment.data.gov.uk/flood-monitoring/id/stations?parameter=rainfall'
    response = requests.get(url)
    eng_station_data = {}
    if response.status_code == 200:
        data = response.json()
        for station in data['items']:
            if station.get('lat') is not None and station.get('long') is not None:
                eng_station_data[station.get('notation')] = {
                    'latitude': station.get('lat'),
                    'longitude': station.get('long')
                }
    return eng_station_data

# Function to fetch rainfall measurements for England
def get_rainfall_data_eng():
    eng_url = "http://environment.data.gov.uk/flood-monitoring/id/measures?parameter=rainfall"
    eng_response = requests.get(eng_url)
    if eng_response.status_code == 200:
        eng_data = eng_response.json()
        return eng_data['items']
    return []

# Function to fetch station data with coordinates for Scotland
def fetch_station_data_sco():
    sco_url = "https://www2.sepa.org.uk/rainfall/api/Stations"
    sco_response = requests.get(sco_url)
    sco_station_data = {}
    if sco_response.status_code == 200:
        sco_stations = sco_response.json()
        for station in sco_stations:
            sco_station_data[station['station_no']] = {
                'latitude': station.get('station_latitude'),
                'longitude': station.get('station_longitude')
            }
    return sco_station_data


# Function to fetch latest hourly rainfall data for Scotland
def get_rainfall_data_sco(station_id):
    sco_url = f"https://www2.sepa.org.uk/rainfall/api/Hourly/{station_id}?all=true"
    sco_response = requests.get(sco_url)
    if sco_response.status_code == 200 and sco_response.json():
        return sco_response.json()[-1]
    return None

# Function to fetch station data with rainfall measurements for Wales
def get_wales_rainfall_data(api_key):
    url = 'https://api.naturalresources.wales/rivers-and-seas/v1/api/StationData'
    headers = {'Ocp-Apim-Subscription-Key': api_key}
    response = requests.get(url, headers=headers)
    wales_rainfall_data = []
    if response.status_code == 200:
        wales_data = response.json()
        for station in wales_data:
            if station['coordinates']['latitude'] is not None and station['coordinates']['longitude'] is not None:
                station_id = station['location']
                latitude = station['coordinates']['latitude']
                longitude = station['coordinates']['longitude']
                rainfall = None
                for parameter in station['parameters']:
                    if parameter['paramNameEN'] == 'Rainfall':
                        rainfall = parameter['latestValue']
                        break
                if rainfall is not None:
                    wales_rainfall_data.append({
                        'station_id': station_id,
                        'rainfall': rainfall,
                        'latitude': latitude,
                        'longitude': longitude
                    })
    return wales_rainfall_data

# Fetching and processing data for England, Scotland, and Wales
eng_station_coordinates = fetch_station_data_eng()
eng_rainfall_data = get_rainfall_data_eng()
sco_station_coordinates = fetch_station_data_sco()
sco_rainfall_data = {station_id: get_rainfall_data_sco(station_id) for station_id in sco_station_coordinates}
wales_rainfall_data = get_wales_rainfall_data('413a14f470f64b70a010cfa3b4ed6a79')  # Replace with actual API key

# Combine the data using latitude and longitude as the key
combined_data = []

# Process and combine England data
for measurement in eng_rainfall_data:
    station_id = measurement.get('stationReference')
    rainfall = measurement.get('latestReading', {}).get('value')
    coordinates = eng_station_coordinates.get(station_id, {'latitude': None, 'longitude': None})
    lat_long_key = (coordinates['latitude'], coordinates['longitude'])
    if coordinates['latitude'] is not None and coordinates['longitude'] is not None:
        combined_data.append([lat_long_key, rainfall, 'England'])


# Process and combine Scotland data
for station_id, coordinates in sco_station_coordinates.items():
    sco_url = f"https://www2.sepa.org.uk/rainfall/api/Hourly/{station_id}?all=true"
    sco_response = requests.get(sco_url)
    if sco_response.status_code == 200 and sco_response.json():
        rainfall_data = sco_response.json()[-1]  # Get the latest reading
        rainfall = rainfall_data['Value']
        lat_long_key = (coordinates['latitude'], coordinates['longitude'])
        if coordinates['latitude'] is not None and coordinates['longitude'] is not None:
            combined_data.append([lat_long_key, rainfall, 'Scotland'])

# Process and combine Wales data
for station_data in wales_rainfall_data:
    rainfall = station_data['rainfall']
    latitude = station_data['latitude']
    longitude = station_data['longitude']
    lat_long_key = (latitude, longitude)
    if latitude is not None and longitude is not None:
        combined_data.append([lat_long_key, rainfall, 'Wales'])


# Update existing CSV with rainfall data
filename = "final.csv"
with open(filename, mode='r', newline='', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    existing_data = list(reader)

# Create a mapping of latitude and longitude to index in existing data
lat_long_to_index = {(float(row['lat']), float(row['long'])): index for index, row in enumerate(existing_data)}

# # Update the rainfall_mm column
# for data_row in combined_data:
#     lat_long_key = data_row[0]
#     rainfall = data_row[1]
#     if lat_long_key in lat_long_to_index and rainfall is not None:
#         existing_data[lat_long_to_index[lat_long_key]]['rainfall_mm'] = rainfall

for data_row in combined_data:
    lat_long_key = data_row[0]
    rainfall = data_row[1]
    print(f"Updating rainfall for coordinates {lat_long_key}: {rainfall} (type: {type(rainfall)})")  # Debugging print
    if lat_long_key in lat_long_to_index and rainfall is not None:
        existing_data[lat_long_to_index[lat_long_key]]['rainfall_mm'] = rainfall


# Write the updated data back to the CSV
with open(filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
    writer.writeheader()
    writer.writerows(existing_data)

# Calculate and update the county_avg
county_rainfall = {}
for row in existing_data:
    county = row['county']
    rainfall = float(row['rainfall_mm']) if row['rainfall_mm'] else 0
    if county not in county_rainfall:
        county_rainfall[county] = {'total_rainfall': 0, 'count': 0}
    county_rainfall[county]['total_rainfall'] += rainfall
    county_rainfall[county]['count'] += 1

for county, data in county_rainfall.items():
    data['average_rainfall'] = "{:.2f}".format(data['total_rainfall'] / data['count']) if data['count'] > 0 else "0.00"

for row in existing_data:
    county = row['county']
    row['county_avg'] = county_rainfall[county]['average_rainfall']

# Write the final updated data back to the CSV
with open(filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
    writer.writeheader()
    writer.writerows(existing_data)

current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Data has been updated in {filename} and current time is {current_time}")


Updating rainfall for coordinates (52.186277, -1.171327): 0.0 (type: <class 'float'>)
Updating rainfall for coordinates (52.986245, -1.163701): None (type: <class 'NoneType'>)
Updating rainfall for coordinates (51.692891, -1.672984): 0.0 (type: <class 'float'>)
Updating rainfall for coordinates (52.493886, -0.649292): None (type: <class 'NoneType'>)
Updating rainfall for coordinates (52.73152, -0.995167): 0.0 (type: <class 'float'>)
Updating rainfall for coordinates (52.635078, -1.944539): 0.2 (type: <class 'float'>)
Updating rainfall for coordinates (52.419334, -1.990391): 0.2 (type: <class 'float'>)
Updating rainfall for coordinates (53.196879, -1.901908): 0.2 (type: <class 'float'>)
Updating rainfall for coordinates (53.287863, -1.579192): 0.0 (type: <class 'float'>)
Updating rainfall for coordinates (52.886657, -2.182004): 0.4 (type: <class 'float'>)
Updating rainfall for coordinates (53.305797, -1.088289): 0.0 (type: <class 'float'>)
Updating rainfall for coordinates (52.564065, -

In [91]:
rainfall

0.2

### Does same thing but without scotland

In [37]:
import requests
import csv
from datetime import datetime

# Function to fetch station data with coordinates for England
def fetch_station_data_eng():
    url = 'https://environment.data.gov.uk/flood-monitoring/id/stations?parameter=rainfall'
    response = requests.get(url)
    eng_station_data = {}
    if response.status_code == 200:
        data = response.json()
        for station in data['items']:
            if station.get('latitude') is not None and station.get('longitude') is not None:
                eng_station_data[station.get('notation')] = {
                    'latitude': station.get('longitude'),
                    'longitude': station.get('long')
                }
    return eng_station_data

# Function to fetch rainfall measurements for England
def get_rainfall_data_eng():
    eng_url = "http://environment.data.gov.uk/flood-monitoring/id/measures?parameter=rainfall"
    eng_response = requests.get(eng_url)
    if eng_response.status_code == 200:
        eng_data = eng_response.json()
        return eng_data['items']
    return []

# Function to fetch station data with rainfall measurements for Wales
def get_wales_rainfall_data(api_key):
    url = 'https://api.naturalresources.wales/rivers-and-seas/v1/api/StationData'
    headers = {'Ocp-Apim-Subscription-Key': api_key}
    response = requests.get(url, headers=headers)
    wales_rainfall_data = []
    if response.status_code == 200:
        wales_data = response.json()
        for station in wales_data:
            if station['coordinates']['latitude'] is not None and station['coordinates']['longitude'] is not None:
                station_id = station['location']
                latitude = station['coordinates']['latitude']
                longitude = station['coordinates']['longitude']
                rainfall = None
                for parameter in station['parameters']:
                    if parameter['paramNameEN'] == 'Rainfall':
                        rainfall = parameter['latestValue']
                        break
                if rainfall is not None:
                    wales_rainfall_data.append({
                        'station_id': station_id,
                        'rainfall': rainfall,
                        'latitude': latitude,
                        'longitude': longitude
                    })
    return wales_rainfall_data

# Fetching and processing data for England, Scotland, and Wales
eng_station_coordinates = fetch_station_data_eng()

# # Print first 10 lines of England station data
# print("First 10 lines of England station data:")
# for i, (station_id, station_info) in enumerate(eng_station_coordinates.items()):
#     if i < 10:
#         print(f"Station ID: {station_id}, Coordinates: {station_info}")
# print()

eng_rainfall_data = get_rainfall_data_eng()

# # Print first 10 lines of England rainfall data
# print("First 10 lines of England rainfall data:")
# for i, measurement in enumerate(eng_rainfall_data[:10]):
#     print(f"Measurement {i+1}: {measurement}")
# print()

wales_rainfall_data = get_wales_rainfall_data('413a14f470f64b70a010cfa3b4ed6a79')

# # Print first 10 lines of Wales rainfall data
# print("First 10 lines of Wales rainfall data:")
# for i, station_data in enumerate(wales_rainfall_data[:10]):
#     print(f"Station ID: {station_data['station_id']}, Rainfall: {station_data['rainfall']} mm")
# print()


# Combine the data using latitude and longitude as the key
combined_data = []

# Process and combine England data
for measurement in eng_rainfall_data:
    station_id = measurement.get('stationReference')
    rainfall = measurement.get('latestReading', {}).get('value')
    coordinates = eng_station_coordinates.get(station_id, {'latitude': None, 'longitude': None})
    lat_long_key = (coordinates['latitude'], coordinates['longitude'])
    if coordinates['latitude'] is not None and coordinates['longitude'] is not None:
        combined_data.append([lat_long_key, rainfall, 'England'])

# Process and combine Wales data
for station_data in wales_rainfall_data:
    rainfall = station_data['rainfall']
    latitude = station_data['latitude']
    longitude = station_data['longitude']
    lat_long_key = (latitude, longitude)
    if latitude is not None and longitude is not None:
        combined_data.append([lat_long_key, rainfall, 'Wales'])

# Update existing CSV with rainfall data
filename = "eng_wales.csv"
with open(filename, mode='r', newline='', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    existing_data = list(reader)

# Create a mapping of latitude and longitude to index in existing data
lat_long_to_index = {(float(row['lat']), float(row['long'])): index for index, row in enumerate(existing_data)}

# Update the rainfall_mm column
for data_row in combined_data:
    lat_long_key = data_row[0]
    rainfall = data_row[1]
    if lat_long_key in lat_long_to_index and rainfall is not None:
        existing_data[lat_long_to_index[lat_long_key]]['rainfall_mm'] = rainfall



# Write the updated data back to the CSV
with open(filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
    writer.writeheader()
    writer.writerows(existing_data)

# Now calculate and update the county_avg
# Calculate county rainfall totals and counts
county_rainfall = {}
for row in existing_data:
    county = row['county']
    rainfall = float(row['rainfall_mm']) if row['rainfall_mm'] else 0
    if county not in county_rainfall:
        county_rainfall[county] = {'total_rainfall': 0, 'count': 0}
    county_rainfall[county]['total_rainfall'] += rainfall
    county_rainfall[county]['count'] += 1

# Calculate averages and update the county_avg column
for county, data in county_rainfall.items():
    data['average_rainfall'] = "{:.2f}".format(data['total_rainfall'] / data['count']) if data['count'] > 0 else "0.00"

for row in existing_data:
    county = row['county']
    row['county_avg'] = county_rainfall[county]['average_rainfall']

# Write the final updated data back to the CSV
with open(filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
    writer.writeheader()
    writer.writerows(existing_data)

current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Data has been updated in {filename} and current time is {current_time}")


Data has been updated in eng_wales.csv and current time is 2024-03-20 00:23:34


In [ ]:
import requests
import pandas as pd
from datetime import datetime

# Function to fetch station data with coordinates for England
def fetch_station_data_eng():
    url = 'https://environment.data.gov.uk/flood-monitoring/id/stations?parameter=rainfall'
    response = requests.get(url)
    eng_station_data = {}
    if response.status_code == 200:
        data = response.json()
        for station in data['items']:
            if station.get('lat') is not None and station.get('long') is not None:
                eng_station_data[station.get('notation')] = {
                    'latitude': station.get('lat'),
                    'longitude': station.get('long')
                }
    return eng_station_data

# Function to fetch rainfall measurements for England
def get_rainfall_data_eng():
    eng_url = "http://environment.data.gov.uk/flood-monitoring/id/measures?parameter=rainfall"
    eng_response = requests.get(eng_url)
    if eng_response.status_code == 200:
        eng_data = eng_response.json()
        return eng_data['items']
    return []

# Function to fetch station data with coordinates for Scotland
def fetch_station_data_sco():
    sco_url = "https://www2.sepa.org.uk/rainfall/api/Stations"
    sco_response = requests.get(sco_url)
    sco_station_data = {}
    if sco_response.status_code == 200:
        sco_stations = sco_response.json()
        for station in sco_stations:
            if station.get('station_latitude') is not None and station.get('station_longitude') is not None:
                sco_station_data[station['station_no']] = {
                    'latitude': station.get('station_latitude'),
                    'longitude': station.get('station_longitude')
                }
    return sco_station_data

# Function to fetch latest hourly rainfall data for Scotland
def get_rainfall_data_sco(station_id):
    sco_url = f"https://www2.sepa.org.uk/rainfall/api/Hourly/{station_id}?all=true"
    sco_response = requests.get(sco_url)
    if sco_response.status_code == 200 and sco_response.json():
        return sco_response.json()[-1]
    return None

# Function to fetch station data with rainfall measurements for Wales
def get_wales_rainfall_data(api_key):
    url = 'https://api.naturalresources.wales/rivers-and-seas/v1/api/StationData'
    headers = {'Ocp-Apim-Subscription-Key': api_key}
    response = requests.get(url, headers=headers)
    wales_rainfall_data = []
    if response.status_code == 200:
        wales_data = response.json()
        for station in wales_data:
            if station['coordinates']['latitude'] is not None and station['coordinates']['longitude'] is not None:
                station_id = station['location']
                latitude = station['coordinates']['latitude']
                longitude = station['coordinates']['longitude']
                
                # Find the 'Rainfall' parameter
                rainfall = None
                for parameter in station['parameters']:
                    if parameter['paramNameEN'] == 'Rainfall':
                        rainfall = parameter['latestValue']
                        break

                # Add the station data to the list if rainfall data is found
                if rainfall is not None:
                    wales_rainfall_data.append({
                        'station_id': station_id,
                        'rainfall': rainfall,
                        'latitude': latitude,
                        'longitude': longitude
                    })
    return wales_rainfall_data

# Fetching and processing data for England, Scotland, and Wales
eng_station_coordinates = fetch_station_data_eng()
eng_rainfall_data = get_rainfall_data_eng()
sco_station_coordinates = fetch_station_data_sco()
sco_rainfall_data = {station_id: get_rainfall_data_sco(station_id) for station_id in sco_station_coordinates}
wales_rainfall_data = get_wales_rainfall_data('413a14f470f64b70a010cfa3b4ed6a79')  # Replace with actual API key

# Combine the data
combined_data = []

# Process and combine England data
for measurement in eng_rainfall_data:
    station_id = measurement.get('stationReference')
    rainfall = measurement.get('latestReading', {}).get('value')
    if station_id in eng_station_coordinates:
        combined_data.append([station_id, rainfall, 'England'])


# Process and combine Wales data
for station_data in wales_rainfall_data:
    station_id = station_data['station_id']
    rainfall = station_data['rainfall']
    if station_data['latitude'] is not None and station_data['longitude'] is not None:
        combined_data.append([station_id, rainfall, 'Wales'])

# Calculate average rainfall for each country
country_rainfall = {'England': {'total_rainfall': 0, 'count': 0},
                    'Scotland': {'total_rainfall': 0, 'count': 0},
                    'Wales': {'total_rainfall': 0, 'count': 0}}

for data_row in combined_data:
    country = data_row[2]  # Corrected index for country
    rainfall = float(data_row[1]) if data_row[1] else 0
    country_rainfall[country]['total_rainfall'] += rainfall
    country_rainfall[country]['count'] += 1

for country, data in country_rainfall.items():
    average_rainfall = round(data['total_rainfall'] / data['count'], 2) if data['count'] > 0 else 0
    country_rainfall[country]['average_rainfall'] = average_rainfall

# Append the country average rainfall to the data rows
for data_row in combined_data:
    country = data_row[2]  # Corrected index for country
    average_rainfall = country_rainfall[country].get('average_rainfall', 0)
    data_row.append(average_rainfall)


# Update final.csv
try:
    final_df = pd.read_csv('final.csv')
    for index, row in final_df.iterrows():
        station_id = row['station_id']
        # Find the corresponding station in combined_data
        for data in combined_data:
            if data[0] == station_id:
                final_df.at[index, 'rainfall_mm'] = data[1]  # Update rainfall amount
                
    # Save the updated dataframe
    final_df.to_csv('final.csv', index=False)
    print("final.csv has been created with updated rainfall data.")
except FileNotFoundError:
    print("final.csv not found. Please make sure the file exists.")

current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Process completed at {current_time}")


In [111]:
import requests
import csv
from datetime import datetime

# Function to safely convert to float
def safe_float_convert(value, default=None):
    try:
        return float(value)
    except (ValueError, TypeError):
        return default

# Function to fetch station data with coordinates for England
def fetch_station_data_eng():
    url = 'https://environment.data.gov.uk/flood-monitoring/id/stations?parameter=rainfall'
    response = requests.get(url)
    eng_station_data = {}
    if response.status_code == 200:
        data = response.json()
        for station in data['items']:
            if station.get('lat') is not None and station.get('long') is not None:
                eng_station_data[station.get('notation')] = {
                    'latitude': station.get('lat'),
                    'longitude': station.get('long')
                }
    return eng_station_data

# Function to fetch rainfall measurements for England
def get_rainfall_data_eng():
    eng_url = "http://environment.data.gov.uk/flood-monitoring/id/measures?parameter=rainfall"
    eng_response = requests.get(eng_url)
    if eng_response.status_code == 200:
        eng_data = eng_response.json()
        return eng_data['items']
    return []

# # Function to fetch station data with coordinates for Scotland
# def fetch_station_data_sco():
#     sco_url = "https://www2.sepa.org.uk/rainfall/api/Stations"
#     sco_response = requests.get(sco_url)
#     sco_station_data = {}
#     if sco_response.status_code == 200:
#         sco_stations = sco_response.json()
#         for station in sco_stations:
#             sco_station_data[station['station_no']] = {
#                 'latitude': station.get('station_latitude'),
#                 'longitude': station.get('station_longitude')
#             }
#     return sco_station_data

# Function to fetch station data with coordinates for Scotland
def fetch_station_data_sco():
    sco_url = "https://www2.sepa.org.uk/rainfall/api/Stations"
    sco_response = requests.get(sco_url)
    sco_station_data = {}
    if sco_response.status_code == 200:
        sco_stations = sco_response.json()
        for station in sco_stations:
            # Convert latitude and longitude to float
            latitude = safe_float_convert(station.get('station_latitude'))
            longitude = safe_float_convert(station.get('station_longitude'))
            if latitude is not None and longitude is not None:
                sco_station_data[station['station_no']] = {
                    'latitude': latitude,
                    'longitude': longitude
                }
    return sco_station_data


# Function to fetch latest hourly rainfall data for Scotland
def get_rainfall_data_sco(station_id):
    sco_url = f"https://www2.sepa.org.uk/rainfall/api/Hourly/{station_id}?all=true"
    sco_response = requests.get(sco_url)
    if sco_response.status_code == 200 and sco_response.json():
        return sco_response.json()[-1]
    return None

# Function to fetch station data with rainfall measurements for Wales
def get_wales_rainfall_data(api_key):
    url = 'https://api.naturalresources.wales/rivers-and-seas/v1/api/StationData'
    headers = {'Ocp-Apim-Subscription-Key': api_key}
    response = requests.get(url, headers=headers)
    wales_rainfall_data = []
    if response.status_code == 200:
        wales_data = response.json()
        for station in wales_data:
            if station['coordinates']['latitude'] is not None and station['coordinates']['longitude'] is not None:
                station_id = station['location']
                latitude = station['coordinates']['latitude']
                longitude = station['coordinates']['longitude']
                rainfall = None
                for parameter in station['parameters']:
                    if parameter['paramNameEN'] == 'Rainfall':
                        rainfall = parameter['latestValue']
                        break
                if rainfall is not None:
                    wales_rainfall_data.append({
                        'station_id': station_id,
                        'rainfall': rainfall,
                        'latitude': latitude,
                        'longitude': longitude
                    })
    return wales_rainfall_data

# Fetching and processing data for England, Scotland, and Wales
eng_station_coordinates = fetch_station_data_eng()
eng_rainfall_data = get_rainfall_data_eng()
sco_station_coordinates = fetch_station_data_sco()
sco_rainfall_data = {station_id: get_rainfall_data_sco(station_id) for station_id in sco_station_coordinates}
wales_rainfall_data = get_wales_rainfall_data('413a14f470f64b70a010cfa3b4ed6a79')  # Replace with actual API key

# Combine the data using latitude and longitude as the key
combined_data = []

# Process and combine England data
for measurement in eng_rainfall_data:
    station_id = measurement.get('stationReference')
    rainfall = safe_float_convert(measurement.get('latestReading', {}).get('value'))
    coordinates = eng_station_coordinates.get(station_id, {'latitude': None, 'longitude': None})
    lat_long_key = (coordinates['latitude'], coordinates['longitude'])
    if coordinates['latitude'] is not None and coordinates['longitude'] is not None:
        combined_data.append([lat_long_key, rainfall, 'England'])

# Process and combine Scotland data
# Process and combine Scotland data
for station_id, coordinates in sco_station_coordinates.items():
    sco_rainfall = get_rainfall_data_sco(station_id)
    if sco_rainfall:
        rainfall = safe_float_convert(sco_rainfall.get('Value'))
        lat_long_key = (coordinates['latitude'], coordinates['longitude'])
        if coordinates['latitude'] is not None and coordinates['longitude'] is not None:
            combined_data.append([lat_long_key, rainfall, 'Scotland'])

# Process and combine Wales data
for station_data in wales_rainfall_data:
    rainfall = safe_float_convert(station_data['rainfall'])
    latitude = station_data['latitude']
    longitude = station_data['longitude']
    lat_long_key = (latitude, longitude)
    if latitude is not None and longitude is not None:
        combined_data.append([lat_long_key, rainfall, 'Wales'])

# Update existing CSV with rainfall data
filename = "coordinates_rainfall_data.csv"
with open(filename, mode='r', newline='', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    existing_data = list(reader)

# Create a mapping of latitude and longitude to index in existing data
lat_long_to_index = {(float(row['lat']), float(row['long'])): index for index, row in enumerate(existing_data)}

# for data_row in combined_data:
#     lat_long_key = data_row[0]
#     rainfall = data_row[1]
#     if lat_long_key in lat_long_to_index and rainfall is not None:
#         existing_data[lat_long_to_index[lat_long_key]]['rainfall_mm'] = rainfall

for data_row in combined_data:
    lat_long_key = data_row[0]
    rainfall = data_row[1]
    if lat_long_key in lat_long_to_index and rainfall is not None:
        existing_data[lat_long_to_index[lat_long_key]]['rainfall_mm'] = rainfall

# Write the updated data back to the CSV
with open(filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
    writer.writeheader()
    writer.writerows(existing_data)

# Calculate and update the county_avg
county_rainfall = {}
for row in existing_data:
    county = row['county']
    rainfall = safe_float_convert(row['rainfall_mm'], default=0)
    if county not in county_rainfall:
        county_rainfall[county] = {'total_rainfall': 0, 'count': 0}
    county_rainfall[county]['total_rainfall'] += rainfall
    county_rainfall[county]['count'] += 1

for county, data in county_rainfall.items():
    data['average_rainfall'] = "{:.2f}".format(data['total_rainfall'] / data['count']) if data['count'] > 0 else "0.00"

for row in existing_data:
    county = row['county']
    row['county_avg'] = county_rainfall[county]['average_rainfall']

# Write the final updated data back to the CSV
with open(filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
    writer.writeheader()
    writer.writerows(existing_data)

current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Data has been updated in {filename} and current time is {current_time}")


Data has been updated in final.csv and current time is 2024-03-20 02:48:15


In [102]:
print(existing_data)

[{'station_id': 'E7050', 'rainfall_mm': 0.0, 'lat': '52.186277', 'long': '-1.171327', 'country': 'England', 'county': 'West Northamptonshire', '': '', 'county_avg': '0.04'}, {'station_id': '4163', 'rainfall_mm': '', 'lat': '52.986245', 'long': '-1.163701', 'country': 'England', 'county': 'England', '': '', 'county_avg': '0.00'}, {'station_id': '0890TH', 'rainfall_mm': 0.0, 'lat': '51.692891', 'long': '-1.672984', 'country': 'England', 'county': 'Gloucestershire', '': '', 'county_avg': '0.03'}, {'station_id': 'E1310', 'rainfall_mm': '', 'lat': '52.493886', 'long': '-0.649292', 'country': 'England', 'county': 'North Northamptonshire', '': '', 'county_avg': '0.06'}, {'station_id': '3680', 'rainfall_mm': 0.0, 'lat': '52.73152', 'long': '-0.995167', 'country': 'England', 'county': 'Leicestershire', '': '', 'county_avg': '0.05'}, {'station_id': '3275', 'rainfall_mm': 0.2, 'lat': '52.635078', 'long': '-1.944539', 'country': 'England', 'county': 'Walsall', '': '', 'county_avg': '0.20'}, {'stat

In [105]:
lat_long_key

(52.657226, -3.1330043)

In [106]:
lat_long_to_index


{(52.186277, -1.171327): 0,
 (52.986245, -1.163701): 1,
 (51.692891, -1.672984): 2,
 (52.493886, -0.649292): 3,
 (52.73152, -0.995167): 4,
 (52.635078, -1.944539): 5,
 (52.419334, -1.990391): 6,
 (53.196879, -1.901908): 7,
 (53.287863, -1.579192): 8,
 (52.886657, -2.182004): 9,
 (53.305797, -1.088289): 10,
 (52.564065, -1.199634): 11,
 (52.47914, -2.975413): 12,
 (50.778033, -2.848812): 366,
 (50.880106, -1.558591): 15,
 (51.807828, -0.03963): 16,
 (51.039914, -0.097897): 17,
 (51.718037, -0.217242): 18,
 (51.185738, -0.408138): 19,
 (52.323011, -0.206231): 20,
 (51.053802, -0.642384): 21,
 (51.730021, 0.040992): 22,
 (51.722932, 0.136235): 23,
 (51.116625, -2.434971): 24,
 (50.516447, -4.209541): 25,
 (53.102452, 0.317508): 26,
 (50.893868, 0.081008): 27,
 (51.727077, -0.10539): 28,
 (54.760082, -3.214413): 29,
 (50.53756, -3.702476): 30,
 (53.70469, -2.135531): 31,
 (52.103622, 0.807145): 32,
 (55.591879, -1.854785): 33,
 (54.105159, -0.642597): 34,
 (51.827809, 0.054118): 35,
 (51.7

In [107]:
print(combined_data)

[[(52.186277, -1.171327), 0.0, 'England'], [(52.986245, -1.163701), None, 'England'], [(51.692891, -1.672984), 0.0, 'England'], [(52.493886, -0.649292), None, 'England'], [(52.73152, -0.995167), 0.0, 'England'], [(52.635078, -1.944539), 0.2, 'England'], [(52.419334, -1.990391), 0.0, 'England'], [(53.196879, -1.901908), 0.0, 'England'], [(53.287863, -1.579192), 0.0, 'England'], [(52.886657, -2.182004), 0.4, 'England'], [(53.305797, -1.088289), 0.0, 'England'], [(52.564065, -1.199634), 0.0, 'England'], [(52.47914, -2.975413), 0.2, 'England'], [(50.778033, -2.848812), 0.0, 'England'], [(50.778033, -2.848812), 0.04, 'England'], [(50.880106, -1.558591), 0.0, 'England'], [(51.807828, -0.03963), 0.0, 'England'], [(51.039914, -0.097897), 0.0, 'England'], [(51.718037, -0.217242), 0.0, 'England'], [(51.185738, -0.408138), 0.0, 'England'], [(52.323011, -0.206231), 0.0, 'England'], [(51.053802, -0.642384), None, 'England'], [(51.730021, 0.040992), 0.0, 'England'], [(51.722932, 0.136235), 0.0, 'Eng

In [108]:
sco_station_coordinates

{'15018': {'latitude': '55.85329633', 'longitude': '-2.387448356'},
 '234150': {'latitude': '57.48021805', 'longitude': '-3.205975095'},
 '115301': {'latitude': '57.26272622', 'longitude': '-5.005979754'},
 '234170': {'latitude': '57.24230294', 'longitude': '-2.717123004'},
 '234151': {'latitude': '58.34717676', 'longitude': '-4.644621014'},
 '115304': {'latitude': '57.28490502', 'longitude': '-6.187008123'},
 '115500': {'latitude': '55.66964373', 'longitude': '-4.413260863'},
 '115501': {'latitude': '55.35009103', 'longitude': '-5.717857841'},
 '15198': {'latitude': '56.06640057', 'longitude': '-3.371633207'},
 '115506': {'latitude': '55.66308049', 'longitude': '-4.757486092'},
 '491811': {'latitude': '57.22063972', 'longitude': '-3.651932222'},
 '15163': {'latitude': '56.11221933', 'longitude': '-4.342412217'},
 '14964': {'latitude': '56.31678388', 'longitude': '-4.112254665'},
 '14952': {'latitude': '56.47302976', 'longitude': '-3.904678161'},
 '234161': {'latitude': '57.50564634', 

In [109]:
eng_station_coordinates

{'E7050': {'latitude': 52.186277, 'longitude': -1.171327},
 '4163': {'latitude': 52.986245, 'longitude': -1.163701},
 '0890TH': {'latitude': 51.692891, 'longitude': -1.672984},
 'E1310': {'latitude': 52.493886, 'longitude': -0.649292},
 '3680': {'latitude': 52.73152, 'longitude': -0.995167},
 '3275': {'latitude': 52.635078, 'longitude': -1.944539},
 '3167': {'latitude': 52.419334, 'longitude': -1.990391},
 '3307': {'latitude': 53.196879, 'longitude': -1.901908},
 '3404': {'latitude': 53.287863, 'longitude': -1.579192},
 '3014': {'latitude': 52.886657, 'longitude': -2.182004},
 '3901': {'latitude': 53.305797, 'longitude': -1.088289},
 '3999': {'latitude': 52.564065, 'longitude': -1.199634},
 '1412': {'latitude': 52.47914, 'longitude': -2.975413},
 'Not_Specified': {'latitude': 50.778033, 'longitude': -2.848812},
 'E13600': {'latitude': 50.880106, 'longitude': -1.558591},
 '242534TP': {'latitude': 51.807828, 'longitude': -0.03963},
 'E8520': {'latitude': 51.039914, 'longitude': -0.097897